In [12]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import csv
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class FinanceTrackerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Personal Finance Tracker")
        self.root.geometry("900x700")

        self.transactions = []
        self.create_widgets()

    def create_widgets(self):
        main_frame = tk.Frame(self.root)
        main_frame.pack(pady=20)

        # Input fields for transactions
        tk.Label(main_frame, text="Amount:").grid(row=0, column=0, padx=10, pady=5)
        self.amount_entry = tk.Entry(main_frame)
        self.amount_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(main_frame, text="Category:").grid(row=1, column=0, padx=10, pady=5)
        self.category_entry = tk.Entry(main_frame)
        self.category_entry.grid(row=1, column=1, padx=10, pady=5)

        tk.Label(main_frame, text="Description:").grid(row=2, column=0, padx=10, pady=5)
        self.description_entry = tk.Entry(main_frame)
        self.description_entry.grid(row=2, column=1, padx=10, pady=5)

        tk.Label(main_frame, text="Date (YYYY-MM-DD):").grid(row=1, column=2, padx=10, pady=5)
        self.date_entry = tk.Entry(main_frame)
        self.date_entry.grid(row=1, column=3, padx=10, pady=5)
        self.date_entry.insert(0, datetime.now().strftime("%Y-%m-%d"))

        # Transaction type selection
        tk.Label(main_frame, text="Transaction Type:").grid(row=0, column=2, padx=10, pady=5)
        self.transaction_type = tk.StringVar(self.root)
        self.transaction_type.set("Income")
        transaction_type_menu = tk.OptionMenu(main_frame, self.transaction_type, "Income", "Expense")
        transaction_type_menu.grid(row=0, column=3, padx=10, pady=5)

        # Month selection for graph sorting
        tk.Label(main_frame, text="Select Month:").grid(row=3, column=2, padx=10, pady=5)
        self.month_selection = ttk.Combobox(main_frame, values=[f"{i:02d}" for i in range(1, 13)], state="readonly")
        self.month_selection.grid(row=3, column=3, padx=10, pady=5)
        self.month_selection.set(datetime.now().strftime("%m"))  # Set current month as default

        # Buttons for operations
        tk.Button(main_frame, text="Add Transaction", command=self.add_transaction).grid(row=3, column=0, padx=10, pady=10)
        tk.Button(main_frame, text="Delete Transaction", command=self.delete_transaction).grid(row=3, column=1, padx=10, pady=10)
        tk.Button(main_frame, text="View Summary", command=self.view_summary).grid(row=4, column=0, padx=10, pady=10)
        tk.Button(main_frame, text="Save Data", command=self.save_data).grid(row=4, column=1, padx=10, pady=10)
        tk.Button(main_frame, text="Sort Data", command=self.sort_data).grid(row=4, column=2, padx=10, pady=10)
        tk.Button(main_frame, text="Show Graphs", command=self.show_graphs).grid(row=5, column=0, columnspan=4, padx=10, pady=10)

        # Listbox for transaction history
        self.transaction_listbox = tk.Listbox(self.root, width=90, height=10)
        self.transaction_listbox.pack(pady=20)

        # Frame for displaying the graphs
        self.graph_frame = tk.Frame(self.root)
        self.graph_frame.pack(pady=20)

    def add_transaction(self):
        try:
            amount = float(self.amount_entry.get())
            transaction_type = self.transaction_type.get()
            if transaction_type == "Expense":
                amount = -abs(amount)
            category = self.category_entry.get()
            description = self.description_entry.get()
            date = self.date_entry.get()
            if not category or not description or not date:
                raise ValueError("Category, description, and date cannot be empty.")
            transaction = {
                'amount': amount,
                'category': category,
                'description': description,
                'date': date
            }
            self.transactions.append(transaction)
            self.amount_entry.delete(0, tk.END)
            self.category_entry.delete(0, tk.END)
            self.description_entry.delete(0, tk.END)
            self.date_entry.delete(0, tk.END)
            self.date_entry.insert(0, datetime.now().strftime("%Y-%m-%d"))
            self.update_transaction_listbox()
            messagebox.showinfo("Success", "Transaction added successfully!")
        except ValueError as e:
            messagebox.showerror("Error", f"Invalid input: {str(e)}")

    def delete_transaction(self):
        selected_index = self.transaction_listbox.curselection()
        if not selected_index:
            messagebox.showerror("Error", "Please select a transaction to delete.")
            return
        self.transactions.pop(selected_index[0])
        self.update_transaction_listbox()

    def update_transaction_listbox(self):
        self.transaction_listbox.delete(0, tk.END)
        for transaction in sorted(self.transactions, key=lambda x: x['date']):
            entry = f"{transaction['date']}: {transaction['category']} - ${transaction['amount']:.2f} ({transaction['description']})"
            self.transaction_listbox.insert(tk.END, entry)

    def view_summary(self):
        total_income = sum(t['amount'] for t in self.transactions if t['amount'] > 0)
        total_expenses = sum(abs(t['amount']) for t in self.transactions if t['amount'] < 0)
        balance = total_income - total_expenses
        summary_message = f"Total Income: ${total_income:.2f}\nTotal Expenses: ${total_expenses:.2f}\nBalance: ${balance:.2f}"
        messagebox.showinfo("Summary", summary_message)

    def save_data(self):
        filepath = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if not filepath:
            return
        with open(filepath, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Date', 'Category', 'Amount', 'Description'])
            for transaction in sorted(self.transactions, key=lambda x: x['date']):  # Save sorted data
                writer.writerow([transaction['date'], transaction['category'], transaction['amount'], transaction['description']])

    def sort_data(self):
        self.transactions.sort(key=lambda x: x['date'])
        self.update_transaction_listbox()

    def show_graphs(self):
        for widget in self.graph_frame.winfo_children():
            widget.destroy()

        selected_month = self.month_selection.get()
        month_transactions = [t for t in self.transactions if t['date'][5:7] == selected_month]

        categories_expenses = {}
        categories_incomes = {}
        for t in month_transactions:
            if t['amount'] < 0:
                categories_expenses[t['category']] = categories_expenses.get(t['category'], 0) + abs(t['amount'])
            else:
                categories_incomes[t['category']] = categories_incomes.get(t['category'], 0) + t['amount']

        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        if categories_expenses:
            ax[0].bar(categories_expenses.keys(), categories_expenses.values(), color='red')
            ax[0].set_title("Expenses by Category for Month " + selected_month)
            ax[0].set_xlabel("Category")
            ax[0].set_ylabel("Amount")

        if categories_incomes:
            ax[1].pie(categories_incomes.values(), labels=categories_incomes.keys(), autopct='%1.1f%%')
            ax[1].set_title("Income Distribution for Month " + selected_month)

        canva = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canva.draw()
        canva.get_tk_widget().pack()

In [14]:
if __name__ == "__main__":
    root = tk.Tk()
    app = FinanceTrackerApp(root)
    root.mainloop()